<a href="https://colab.research.google.com/github/hap6v6/ML_study/blob/main/%EC%B6%94%EC%B2%9C%20%EC%8B%9C%EC%8A%A4%ED%85%9C%20%EC%9E%85%EB%AC%B8%ED%8E%B8/5_content_based_recommendation_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Content-Based Recommendation 1

In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

# Read Data

In [2]:
movies = pd.read_csv('/content/drive/MyDrive/추천 시스템 입문(inflearn)/data/ml-latest-small/movies.csv')
ratings = pd.read_pickle('/content/drive/MyDrive/추천 시스템 입문(inflearn)/data/ml-latest-small/ratings_updated.p')
genres = pd.read_pickle('/content/drive/MyDrive/추천 시스템 입문(inflearn)/data/ml-latest-small/genres.p')

# Preprocessing (전처리)

In [3]:
# ratings = ratings + genres
# 기준 설정: left_on='movieId' (왼쪽 테이블에서는 movieId 기준으로), right_index=True (오른쪽 테이블에서는 인덱스 기준으로)
ratings = ratings.merge(genres, left_on='movieId', right_index=True)
ratings.sample()

,userId,movieId,rating,timestamp,(no genres listed),Action,Adventure,Animation,Children,Comedy,...,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
8957,62,53125,4.5,2018-03-19 20:07:04,0,1,1,0,0,1,...,0,0,0,0,0,0,0,0,0,0


In [4]:
# 장르 데이터 null 값 표시 변경: 0으로 표시 -> NaN으로 표시 (후에 계산시 편함)
ratings = ratings.replace(0, np.nan)
ratings.sample()

,userId,movieId,rating,timestamp,(no genres listed),Action,Adventure,Animation,Children,Comedy,...,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
53072,351,1721,4.0,2012-01-09 10:56:51,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN


# Train Test Split

In [5]:
from sklearn.model_selection import train_test_split

In [6]:
train, test = train_test_split(ratings, random_state = 42, test_size=0.1)

In [7]:
print(train.shape)
print(test.shape)

(90765, 24)
(10085, 24)


# Item profile

In [8]:
genres.loc[4896]
# 이와 같은 영화의 장르를 아이템 프로필이라고 생각하면 됨.

(no genres listed)    0
Action                0
Adventure             1
Animation             0
Children              1
Comedy                0
Crime                 0
Documentary           0
Drama                 0
Fantasy               1
Film-Noir             0
Horror                0
IMAX                  0
Musical               0
Mystery               0
Romance               0
Sci-Fi                0
Thriller              0
War                   0
Western               0
Name: 4896, dtype: int64

# User profile
- 각 장르에 대해서 유저가 매긴 평점의 평균
- train 데이터 사용해서 연산해줘야 함

In [9]:
genre_cols = genres.columns
genre_cols

Index(['(no genres listed)', 'Action', 'Adventure', 'Animation', 'Children',
       'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy', 'Film-Noir',
       'Horror', 'IMAX', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller',
       'War', 'Western'],
      dtype='object')

In [10]:
# 각 영화 평점 * 장르 정보
for cols in genre_cols:
  train[cols] = train[cols] * train['rating']

In [11]:
train.head()

,userId,movieId,rating,timestamp,(no genres listed),Action,Adventure,Animation,Children,Comedy,...,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
36903,249,52281,4.0,2014-12-06 20:21:19,NaN,4.0,NaN,NaN,NaN,NaN,...,NaN,4.0,NaN,NaN,NaN,NaN,4.0,4.0,NaN,NaN
53734,354,3996,4.0,2008-01-20 23:03:24,NaN,4.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN
52124,339,4718,3.5,2016-04-11 03:38:25,NaN,NaN,NaN,NaN,NaN,3.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
69642,448,26386,3.5,2006-01-07 16:39:43,NaN,NaN,NaN,NaN,NaN,3.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.5,NaN,NaN
100412,610,72142,5.0,2016-11-19 08:50:13,NaN,5.0,NaN,NaN,NaN,5.0,...,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN


In [12]:
# ex. 249번 유저가 Action 장르에 평균적으로 어떤 평점을 매겼는가
# 각 user 별로, Action 아이템에 준 평점의 평균
train.groupby('userId')['Action'].mean()

userId
1       4.303797
2       4.055556
3       3.571429
4       3.272727
5       3.250000
          ...   
608     3.325397
609     3.090909
610     3.607066
1000    4.250000
1001    4.500000
Name: Action, Length: 612, dtype: float64

In [13]:
# 위의 내용 모든 장르에 적용
user_profile = train.groupby('userId')[genre_cols].mean()
user_profile.head()
# 1번 유저는 Adventure 장르의 영화에 대해서는 평균적으로 4.36, Animation에 대해서는 4.68점을 주는 사람이다.

,(no genres listed),Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
userId,,,,,,,,,,,,,,,,,,,,
1,NaN,4.303797,4.368421,4.689655,4.550000,4.287671,4.333333,NaN,4.583333,4.292683,NaN,3.250000,NaN,4.736842,4.214286,4.300000,4.250000,4.152174,4.473684,4.2
2,NaN,4.055556,4.166667,NaN,NaN,4.200000,3.777778,4.333333,3.882353,NaN,NaN,NaN,3.750000,NaN,4.000000,4.500000,3.875000,3.666667,4.500000,3.5
3,NaN,3.571429,2.727273,0.500000,0.500000,1.000000,0.500000,NaN,0.833333,4.000000,NaN,4.642857,NaN,0.500000,5.000000,0.500000,4.200000,4.142857,0.500000,NaN
4,NaN,3.272727,3.730769,4.000000,3.800000,3.568421,3.769231,4.000000,3.446429,3.833333,4.0,4.250000,3.000000,4.000000,3.428571,3.407407,2.818182,3.441176,3.571429,3.5
5,NaN,3.250000,3.428571,4.333333,4.111111,3.545455,3.818182,NaN,3.782609,4.333333,NaN,3.000000,3.666667,4.400000,NaN,3.000000,2.500000,3.714286,3.333333,3.0


## My user profile

In [14]:
user_profile.loc[1000]

(no genres listed)         NaN
Action                4.250000
Adventure             3.500000
Animation             2.000000
Children                   NaN
Comedy                3.333333
Crime                      NaN
Documentary                NaN
Drama                 4.333333
Fantasy               2.000000
Film-Noir                  NaN
Horror                     NaN
IMAX                       NaN
Musical               2.000000
Mystery                    NaN
Romance               2.000000
Sci-Fi                4.250000
Thriller                   NaN
War                        NaN
Western                    NaN
Name: 1000, dtype: float64

# Predict

## 샘플 유저의 평점 예측

In [15]:
# 임의의 13852번을 샘플 데이터로 사용
sample = test.loc[13852]

In [16]:
# sample user의 user profile 가져오기
sample_user = sample['userId']
sample_user_profile = user_profile.loc[sample_user]
sample_user_profile
# Horror, Western은 좋아하는 편이고 Action, Romance는 별로 안 좋아하는 성향을 가진 유저다

(no genres listed)    3.000000
Action                3.392857
Adventure             3.512500
Animation             4.000000
Children              3.883333
Comedy                3.471526
Crime                 3.329268
Documentary           4.333333
Drama                 3.382979
Fantasy               3.437500
Film-Noir                  NaN
Horror                4.212121
IMAX                  4.000000
Musical               3.052632
Mystery               3.708333
Romance               3.131068
Sci-Fi                3.383333
Thriller              3.868421
War                   3.318182
Western               4.500000
Name: 89, dtype: float64

In [17]:
# 유저가 본 영화 조회
print(sample['movieId'])
# [genre_cols]: Action, Adventure ... 등 genre_cols 리스트의 값 모두 넣음
sample[genre_cols]

88069


(no genres listed)    NaN
Action                NaN
Adventure             NaN
Animation             NaN
Children              NaN
Comedy                1.0
Crime                 1.0
Documentary           NaN
Drama                 NaN
Fantasy               NaN
Film-Noir             NaN
Horror                NaN
IMAX                  NaN
Musical               NaN
Mystery               NaN
Romance               NaN
Sci-Fi                NaN
Thriller              NaN
War                   NaN
Western               NaN
Name: 13852, dtype: object

In [18]:
# 유저가 본 영화 조회
movies[movies['movieId'] == 88069]
# 유저가 본 영화인 Delhi Belly의 장르 Comedy, Crime에 유저는 평균적으로 3.47, 3.40점을 매겼다.
# 그렇다면 유저는 Delhi Belly에 이 두 값의 평균으로 평점을 매기지 않았을까? <예측>

,movieId,title,genres
7641,88069,Delhi Belly (2011),Comedy|Crime


In [19]:
# 위의 내용 연산 과정
(sample_user_profile * sample[genre_cols]).mean()
# 예측 알고리즘의 결과: 샘플로 뽑은 89번 유저는 88069 영화(Delhi Belly)에 대해서 3.4점을 주었을 것이라 예측한다.

3.4003972442913497

## 전체 유저의 평점 예측과 모델 평가
전체 데이터로 확장

In [20]:
# for문의 진행 사항 시각적으로 보여주는 패키지
from tqdm import tqdm_notebook

In [21]:
# iterrows(): 각 행의 정보를 담은 객체 (index, row_series)
# 행 별로 연산 반복
predict = []
for idx, row in tqdm_notebook(test.iterrows()): 
  userId = row['userId']
  # user profile * item profile
  predict.append((user_profile.loc[userId] * row[genre_cols]).mean())

0it [00:00, ?it/s]

In [22]:
# 예측 값 테스트 셋에 저장
test['predict'] = predict

In [23]:
# null 값 확인
test[test['predict'].isnull()]
# ex. index 14091: train 데이터에 들어가 있는 90번 유저는 Horror 장르의 영화를 한 번도 본 적이 없다.
# 유저가 완전히 새로운 장르의 영화를 볼 때, 예측할 수가 없다. -> Cold-start 문제
# 그렇지만 어떻게든 NaN 대신 합리적인 예측 값을 보여줘야 함 -> 여기서는 global mean (or 유저 평점 평균)

,userId,movieId,rating,timestamp,(no genres listed),Action,Adventure,Animation,Children,Comedy,...,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western,predict
51243,331,5669,2.5,2018-09-17 04:18:22,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
48086,312,1289,5.0,2003-01-21 19:26:40,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
44460,296,169034,4.5,2018-07-30 23:40:43,1.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14091,90,735,4.0,1997-02-19 12:12:09,NaN,NaN,NaN,NaN,NaN,NaN,...,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
22163,147,596,3.0,2008-02-17 17:02:10,NaN,NaN,NaN,1.0,1.0,NaN,...,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5400,38,246,5.0,1996-08-29 18:01:55,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
30716,215,60766,5.0,2009-12-15 20:28:49,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
79767,496,904,5.0,2014-11-05 05:50:05,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,1.0,NaN,NaN,1.0,NaN,NaN,NaN
65294,418,137337,4.0,2016-04-28 18:22:58,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [24]:
# (직접 확인) train 데이터에 들어가 있는 90번 유저는 Horror 장르의 영화를 한 번도 본 적이 없다.
# not null 총합 개수 = 0
train['Horror'][train['userId'] == 90].notnull().sum()

0

In [25]:
# Cold-start 문제 해결: NaN 값을 global mean으로 채워주자
test.loc[test['predict'].isnull(), 'predict'] = train['rating'].mean()

In [26]:
# 다시 null 값 확인
test[test['predict'].isnull()]

,userId,movieId,rating,timestamp,(no genres listed),Action,Adventure,Animation,Children,Comedy,...,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western,predict


### Model validation

In [27]:
from sklearn.metrics import mean_squared_error

In [28]:
mse = mean_squared_error(test['rating'], test['predict'])
rmse = np.sqrt(mse)
rmse
# 전체 평균이나, 특점 유저의 평균, 특정 아이템의 평균 등으로 예측 하는 것보다 
# 개인의 profile을 기반으로 추천해주는 것(=개인의 추천)이 조금 더 정확하다

0.9183601408335907

추가로 해볼 수 있는 것 (sec8. 나와 친구를 위한 영화 추천)
- 나의 user profile로 성향 살펴보기
- 나의 user profile 기반으로 나에게 영화 추천하기